# Strands Agents와 Ollama 모델을 사용한 로컬 에이전트 구축

이 노트북은 Strands Agent와 Ollama를 사용하여 개인 에이전트를 생성하는 방법을 보여줍니다. 에이전트는 파일 작업, 웹 검색, 시스템 명령과 같은 다양한 로컬 작업을 수행할 수 있습니다.

## Ollama란 무엇인가요?

[Ollama](https://ollama.com/)는 머신에서 대규모 언어 모델(LLM)을 로컬로 실행할 수 있게 해주는 오픈소스 프레임워크입니다. 이러한 모델과 상호작용하기 위한 간단한 API를 제공하므로, 외부 서비스로 데이터를 보내고 싶지 않은 프라이버시 중심 애플리케이션에 이상적입니다.

Ollama의 주요 이점:
- **프라이버시**: 모든 처리가 머신에서 로컬로 수행됩니다
- **API 비용 없음**: 원하는 만큼 무료로 사용할 수 있습니다
- **오프라인 기능**: 인터넷 연결 없이도 작동합니다
- **사용자 정의**: 특정 용도에 맞게 미세 조정할 수 있습니다


## 에이전트 세부사항

<div style="float: left; margin-right: 20px;">
    
|기능                |설명                                               |
|--------------------|---------------------------------------------------|
|사용된 기능          |Ollama 모델 - 파일 작업 에이전트 생성               |
|에이전트 구조        |단일 에이전트 아키텍처                              |


</div>


### 에이전트 아키텍처

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [ ]:
%pip install -r requirements.txt

## 설정 및 설치

이 노트북을 실행하기 전에 다음을 확인하세요:

1. Ollama 설치: [https://ollama.com/download](https://ollama.com/download)의 지침을 따르세요
2. Ollama 서버 시작: `ollama serve`
3. Ollama로 모델 다운로드: `ollama pull llama3.2:1b`

자세한 지침은 [문서](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/)를 참조하세요.

이 노트북에서는 SageMaker Studio와의 호환성을 위해 Linux 배포판용 Ollama를 다운로드합니다. 이는 Workshop Studio에서 AWS 주도 워크샵 중 코드 실행을 위해 수행됩니다. 로컬에서 이 코드를 실행하는 경우 현재 환경에 맞게 Ollama 다운로드 단계를 조정해야 합니다.

In [ ]:
# 이것은 Linux 컴퓨터에서 작동합니다
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

In [ ]:
!ollama pull llama3.2:1b

In [ ]:
# 필요한 라이브러리 가져오기
import os

import requests

# strands 구성 요소 가져오기
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [ ]:
# Ollama가 실행 중인지 확인:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama가 실행 중입니다. 사용 가능한 모델:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama가 실행되지 않습니다. 계속하기 전에 Ollama를 시작하세요.")

## 사용자 정의 도구 정의

도구는 에이전트가 환경과 상호작용하는 데 사용할 수 있는 함수입니다. 아래에서는 개인 에이전트를 위한 몇 가지 유용한 도구를 정의합니다.

In [ ]:
# 파일 작업 도구


@tool
def file_read(file_path: str) -> str:
    """파일을 읽고 내용을 반환합니다.

    Args:
        file_path (str): 읽을 파일의 경로

    Returns:
        str: 파일의 내용

    Raises:
        FileNotFoundError: 파일이 존재하지 않는 경우
    """
    try:
        with open(file_path, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"오류: 파일 '{file_path}'을(를) 찾을 수 없습니다."
    except Exception as e:
        return f"파일 읽기 오류: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """파일에 내용을 씁니다.

    Args:
        file_path (str): 파일의 경로
        content (str): 파일에 쓸 내용

    Returns:
        str: 성공 또는 실패를 나타내는 메시지
    """
    try:
        # 디렉토리가 존재하지 않으면 생성
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"파일 '{file_path}'이(가) 성공적으로 작성되었습니다."
    except Exception as e:
        return f"파일 쓰기 오류: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """지정된 경로의 파일과 디렉토리를 나열합니다.

    Args:
        directory_path (str): 나열할 디렉토리의 경로

    Returns:
        str: 모든 파일과 디렉토리를 나열하는 형식화된 문자열
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"폴더: {item}/")
            else:
                files.append(f"파일: {item}")

        result = f"{os.path.abspath(directory_path)}의 내용:\n"
        result += (
            "\n디렉토리:\n" + "\n".join(sorted(directories))
            if directories
            else "\n디렉토리를 찾을 수 없습니다."
        )
        result += (
            "\n\n파일:\n" + "\n".join(sorted(files)) if files else "\n파일을 찾을 수 없습니다."
        )

        return result
    except Exception as e:
        return f"디렉토리 나열 오류: {str(e)}"

## Ollama 기반 에이전트 생성

이제 Ollama 모델과 위에서 정의한 도구를 사용하여 에이전트를 생성하겠습니다.

참고: `execute_commands`, `search_files` 등과 같은 더 많은 도구를 추가할 수 있습니다.

In [ ]:
# 에이전트를 위한 포괄적인 시스템 프롬프트 정의
system_prompt = """
당신은 사용자를 위해 로컬 파일 작업과 간단한 작업을 수행할 수 있는 도움이 되는 개인 어시스턴트입니다.

주요 기능:
1. 파일을 읽고, 이해하고, 요약합니다.
2. 파일을 생성하고 작성합니다.
3. 디렉토리 내용을 나열하고 파일에 대한 정보를 제공합니다.
4. 텍스트 내용을 요약합니다

도구 사용 시:
- 작업 전에 항상 파일 경로를 확인하세요
- 시스템 명령을 사용할 때 주의하세요
- 수행하는 작업에 대한 명확한 설명을 제공하세요
- 작업을 완료할 수 없는 경우 이유를 설명하고 대안을 제안하세요

항상 도움이 되고, 간결하며, 사용자의 요구를 효율적으로 해결하는 데 집중하세요.
"""

model_id = (
    "llama3.2:1b"  # Ollama로 가져온 모든 모델로 변경할 수 있습니다.
)

#### Ollama 모델 구성
Ollama 서비스가 http://localhost:11434에서 실행 중이고 `model_id`가 위에 출력된 Ollama 모델 목록에 있는지 확인하세요.

In [ ]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    params={
        "max_tokens": 4096,  # 모델의 기능에 따라 조정
        "temperature": 0.7,  # 더 결정적인 응답을 위해 낮게, 더 창의적인 응답을 위해 높게
        "top_p": 0.9,  # 핵 샘플링 매개변수
        "stream": True,  # 스트리밍 응답 활성화
    },
)

# 에이전트 생성
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## 에이전트 테스트

몇 가지 예제 작업으로 에이전트를 테스트해보겠습니다.

In [ ]:
local_agent(
    "`sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` 경로의 파일을 읽고 5개의 요점으로 요약해주세요."
)

In [ ]:
# 예제 2: 현재 디렉토리의 파일 나열
response = local_agent("현재 디렉토리의 파일들을 보여주세요")

In [ ]:
# 예제 3: 샘플 파일 생성
response = local_agent(
    "'sample.txt'라는 파일을 '이것은 내 Ollama 에이전트가 만든 테스트 파일입니다.'라는 내용으로 생성해주세요."
)

In [ ]:
# 예제 4: 여러 파일을 읽고 이해한 후 readme 파일 생성
response = local_agent("현재 디렉토리에 대한 readme.md를 생성해주세요")

## 결론

이 노트북에서는 Strands와 Ollama를 사용하여 로컬 개인 에이전트를 만들었습니다. 에이전트는 파일 작업(읽기, 쓰기, 추가)과 텍스트 요약/분석을 수행할 수 있습니다.

이것은 Ollama로 AI 모델을 로컬에서 실행하는 것의 강력함과 strands의 도구 시스템의 유연성을 보여줍니다. 필요에 따라 더 많은 도구를 추가하거나 다른 Ollama 모델을 사용하여 이 에이전트를 확장할 수 있습니다.

### 다음 단계 (아이디어)

- 다양한 Ollama 모델을 시도하여 에이전트의 기능에 어떤 영향을 미치는지 확인해보세요
- 웹 검색, 이메일 전송 또는 캘린더 통합과 같은 더 복잡한 도구를 추가하세요
- 에이전트가 과거 상호작용을 기억할 수 있도록 메모리를 구현하세요
- 에이전트와 상호작용하기 위한 간단한 UI를 만드세요